## Import Libraries

In [ ]:
import math
import soundfile
import numpy as np
from statistics import mean
import IPython.display as ipd
from IPython.display import Audio

## Define WAV File Input Path

In [ ]:
# Load the WAV file
data, sample_rate = soundfile.read("Input.wav")

## Set Parameters

In [ ]:
# Set the RMS threshold and attack/release times
rms_threshold = 0.05  # Currently difficult to set due log nature of scale
attack = 0.26  # Attack time in seconds
release = 0.22  # Release time in seconds

## Run Silence Removal

In [ ]:
# Quick make data mono if stereo
if len(data.shape) == 2:
    data = data[:, 0]

# Create a list to store the modified audio data
modified_data = []

# Calculate the RMS value for every x of audio
window_size = int(sample_rate/4)  # 1/4 second
attack_time = int(attack * sample_rate)  # Attack time in number of samples
release_time = int(release * sample_rate)  # Release time in number of samples
attack_count = 0  # Counter for attack time
release_count = 0  # Counter for release time

for i in range(0, len(data), window_size):
    window = data[i:i+window_size]
    rms = math.sqrt(mean(x ** 2 for x in window))
    if rms >= rms_threshold:
        # Add the window to the modified data if the RMS value is above the threshold
        modified_data.extend(window)
        attack_count = attack_time  # Reset the attack count
        release_count = 0  # Reset the release count
    elif attack_count > 0:
        # Add the window to the modified data for the duration of the attack time
        modified_data.extend(window)
        attack_count -= window_size
    elif release_count < release_time:
        # Add the window to the modified data for the duration of the release time
        modified_data.extend(window)
        release_count += window_size

# Write the modified audio data to a new WAV file
soundfile.write("Output.wav", modified_data, sample_rate)

## Play Output in Notebook

In [ ]:
Audio("Output.wav", rate=sample_rate, autoplay=False)